# **Installation and Imports**

In [ ]:
!pip3 install einops

In [ ]:
%matplotlib inline
import numpy as np
from PIL import Image
from tqdm import tqdm
from copy import deepcopy
from typing import Callable
from itertools import repeat
from functools import partial
import matplotlib.pyplot as plt
from collections import OrderedDict
from IPython.display import clear_output
import torch.utils.model_zoo as model_zoo
import os, math,h5py, imageio, warnings, logging, collections.abc, cv2

import torch
import torch.nn as nn
from einops import rearrange
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

warnings.filterwarnings("ignore")
_logger = logging.getLogger(__name__)

# <center><font size="+6"> **1) Building the model**</font>

## **Config Setup**

In [ ]:
def _cfg(url='', **kwargs):
    return {
        'url': url,
        'num_classes': 1000, 'input_size': (3, 224, 224), 'pool_size': None,
        'crop_pct': .9, 'interpolation': 'bicubic',
        'first_conv': 'patch_embed.proj', 'classifier': 'head',
        **kwargs
    }


model_base_dir = "/kaggle/input/popular-vits-for-interpretation/pytorch/default/1"
default_config = {
    # patch models
    'vit_small_patch16_224': _cfg(
        url=f'{model_base_dir}/vit_small_p16_224-15ec54c9.pth',
    ),
    'vit_base_patch16_224': _cfg(
        url=f'{model_base_dir}/jx_vit_base_p16_224-80ecf9dd.pth',
        mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5),
    ),
    'vit_base_patch16_224_moco': _cfg(
        url=f'{model_base_dir}/linear-vit-b-300ep.pth.tar',
        mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5),
    ),
    'vit_base_patch16_224_dino': _cfg(
        url_backbone=f'{model_base_dir}/dino_vitbase16_pretrain.pth',
        url_linear=f'{model_base_dir}/dino_vitbase16_linearweights.pth',
        url=f'{model_base_dir}/dino_vitbase16_pretrain_full_checkpoint.pth',
        mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5),
    ),
    'vit_mae_patch16_224': _cfg(
        url=f'{model_base_dir}/mae_finetuned_vit_base.pth',
        mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5),
    ),
    'vit_large_patch16_224': _cfg(
        url=f'{model_base_dir}/jx_vit_large_p16_224-4ee7a4dc.pth',
        mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
}

cls = ['airplane', 'bicycle', 'bird',
       'boat',
       'bottle',
       'bus',
       'car',
       'cat',
       'chair',
       'cow',
       'dining table',
       'dog',
       'horse',
       'motobike',
       'person',
       'potted plant',
       'sheep',
       'sofa',
       'train',
       'tv'
       ]

# **Image Loader**

In [ ]:
class ImagenetSegLoader(data.Dataset):
    CLASSES = 2

    def __init__(self, path, length_limit=None, transform=None, target_transform=None):
        self.path = path
        self.transform = transform
        self.target_transform = target_transform
        self.h5py = None
        tmp = h5py.File(path, 'r')

        if length_limit:
            self.data_length = length_limit
        else:
            self.data_length = len(tmp['/value/img'])

        tmp.close()
        del tmp

    def __getitem__(self, index):

        if self.h5py is None:
            self.h5py = h5py.File(self.path, 'r')

        img = np.array(self.h5py[self.h5py['/value/img'][index, 0]]).transpose((2, 1, 0))
        target = np.array(self.h5py[self.h5py[self.h5py['/value/gt'][index, 0]][0, 0]]).transpose((1, 0))

        img = Image.fromarray(img).convert('RGB')
        target = Image.fromarray(target)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = np.array(self.target_transform(target)).astype('int32')
            target = torch.from_numpy(target).long()

        return img, target

    def __len__(self):
        return self.data_length


def image_vizformat(image):
    inv_normalize = transforms.Normalize(
        mean=[-0.5 / 0.5, -0.5 / 0.5, -0.5 / 0.5],
        std=[1 / 0.5, 1 / 0.5, 1 / 0.5]
    )

    img = inv_normalize(image[0])
    img = torch.permute(img, (1, 2, 0))
    img = img.detach().cpu().numpy()
    return img


## **Pretrained Models loader function**

In [ ]:
"""
    
    @@ From load_pretrained.py

    Load pretrained model.
     
     For dino, the url_linear contain weight an bias for the head that needed to be injected into backbone. 
     Cause, the backbone doesn't have a classifier. Instead, is a very good feature extractor.
     
     No other model type require this processing.
     
"""


def load_pretrained(model, cfg=None, num_classes=1000, in_chans=3, filter_fn=None, strict=True, mae=False, moco=False, dino=False):
    if cfg is None:
        cfg = getattr(model, 'default_cfg')

    if dino:
        state_backbone = torch.load(cfg['url_backbone'], map_location='cpu')
        state_linear = torch.load(cfg['url_linear'], map_location='cpu')[
            'state_dict']  # Get weight of the last layer only.
        state_dict = state_backbone.copy()
        state_dict['head.weight'] = state_linear[
            'module.linear.weight']  # Shape: (1000, 1536). That means, the final layer would have input from previous fc layer with 1536 neurons.
        state_dict['head.bias'] = state_linear['module.linear.bias']  # Shape: (1000,)
    else:
        state_dict = torch.load(cfg['url'], map_location='cpu')

    if mae:
        state_dict = state_dict['model']
    if moco:
        state_dict = state_dict['state_dict']
        for i in list(state_dict.keys()):
            name = i.split('module.')[1]
            state_dict[name] = state_dict.pop(i)

    if filter_fn is not None:
        state_dict = filter_fn(state_dict)

    if in_chans == 1:
        conv1_name = cfg['first_conv']
        _logger.info('Converting first conv (%s) pretrained weights from 3 to 1 channel' % conv1_name)
        conv1_weight = state_dict[conv1_name + '.weight']
        # Some weights are in torch.half, ensure its float for sum on CPU
        conv1_type = conv1_weight.dtype
        conv1_weight = conv1_weight.float()
        O, I, J, K = conv1_weight.shape
        if I > 3:
            assert conv1_weight.shape[1] % 3 == 0
            conv1_weight = conv1_weight.reshape(O, I // 3, 3, J, K)  # For models with space2depth stems
            conv1_weight = conv1_weight.sum(dim=2, keepdim=False)
        else:
            conv1_weight = conv1_weight.sum(dim=1, keepdim=True)
        conv1_weight = conv1_weight.to(conv1_type)
        state_dict[conv1_name + '.weight'] = conv1_weight
    elif in_chans != 3:
        conv1_name = cfg['first_conv']
        conv1_weight = state_dict[conv1_name + '.weight']
        conv1_type = conv1_weight.dtype
        conv1_weight = conv1_weight.float()
        O, I, J, K = conv1_weight.shape
        if I != 3:
            _logger.warning('Deleting first conv (%s) from pretrained weights.' % conv1_name)
            del state_dict[conv1_name + '.weight']
            strict = False
        else:
            # NOTE this strategy should be better than random init, but there could be other combinations of
            # the original RGB input layer weights that'd work better for specific cases.
            _logger.info('Repeating first conv (%s) weights in channel dim.' % conv1_name)
            repeat = int(math.ceil(in_chans / 3))
            conv1_weight = conv1_weight.repeat(1, repeat, 1, 1)[:, :in_chans, :, :]
            conv1_weight *= (3 / float(in_chans))
            conv1_weight = conv1_weight.to(conv1_type)
            state_dict[conv1_name + '.weight'] = conv1_weight

    classifier_name = cfg['classifier']
    if num_classes == 1000 and cfg['num_classes'] == 1001:  # special case for imagenet trained models with extra background class in pretrained weights
        classifier_weight = state_dict[classifier_name + '.weight']
        state_dict[classifier_name + '.weight'] = classifier_weight[
                                                  1:]  # Class index 0 give to background. Therefore, starting from index 1.
        classifier_bias = state_dict[classifier_name + '.bias']
        state_dict[classifier_name + '.bias'] = classifier_bias[1:]
    elif num_classes != cfg['num_classes']:
        # completely discard fully connected for all other differences between pretrained and created model
        del state_dict[classifier_name + '.weight']
        del state_dict[classifier_name + '.bias']
        strict = False

    model.load_state_dict(state_dict, strict=strict)


"""
    Load state dictionary of the pretrained model. 

"""


def load_state_dict(checkpoint_path, use_ema=False):
    if checkpoint_path and os.path.isfile(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        state_dict_key = 'state_dict'
        if isinstance(checkpoint, dict):
            if use_ema and 'state_dict_ema' in checkpoint:
                state_dict_key = 'state_dict_ema'
        if state_dict_key and state_dict_key in checkpoint:
            new_state_dict = OrderedDict()
            for k, v in checkpoint[state_dict_key].items():
                # strip `module.` prefix
                name = k[7:] if k.startswith('module') else k
                new_state_dict[name] = v
            state_dict = new_state_dict
        else:
            state_dict = checkpoint
        _logger.info("Loaded {} from checkpoint '{}'".format(state_dict_key, checkpoint_path))
        return state_dict
    else:
        _logger.error("No checkpoint found at '{}'".format(checkpoint_path))
        raise FileNotFoundError()

        
  
"""

    @@@@ From model_loaders.py

"""

def _conv_filter(state_dict, patch_size=16):
    """ convert patch embedding weight from manual patchify + linear proj to conv"""
    out_dict = {}
    for k, v in state_dict.items():
        if 'patch_embed.proj.weight' in k:
            v = v.reshape((v.shape[0], 3, patch_size, patch_size))
        out_dict[k] = v
    return out_dict


def vit_small_patch16_224(pretrained=False, w_rel=False, **kwargs):
    if w_rel:
        model = VisionTransformerWRel(patch_size=16, embed_dim=768, depth=8, num_heads=6, mlp_ratio=3, qkv_bias=False,
                                      **kwargs)
    else:
        model = VisionTransformer(patch_size=16, embed_dim=768, depth=8, num_heads=6, mlp_ratio=3, qkv_bias=False,
                                  norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)

    model.default_cfg = default_config['vit_small_patch16_224']
    if pretrained:
        load_pretrained(
            model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3), filter_fn=_conv_filter)
    return model


def vit_base_patch16_224(pretrained=False, w_rel=False, **kwargs):
    if w_rel:
        model = VisionTransformerWRel(patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
                                      **kwargs)
    else:
        model = VisionTransformer(patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
                                  norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    model.default_cfg = default_config['vit_base_patch16_224']
    if pretrained:
        load_pretrained(
            model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3), filter_fn=_conv_filter)
    return model


def vit_base_patch16_224_dino(pretrained=False, w_rel=False, **kwargs):
    if w_rel:
        model = VisionTransformerWRel(patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
                                      dino=True, **kwargs)
    else:
        model = VisionTransformer(patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
                                  norm_layer=partial(nn.LayerNorm, eps=1e-6), dino=True, **kwargs)

    model.default_cfg = default_config['vit_base_patch16_224_dino']
    if pretrained:
        load_pretrained(
            model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3), filter_fn=_conv_filter, dino=True)
    return model


def vit_base_patch16_224_moco(pretrained=False, w_rel=False, **kwargs):
    if w_rel:
        model = VisionTransformerWRel(patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
                                      **kwargs)
    else:
        model = VisionTransformer(patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
                                  norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)

    model.default_cfg = default_config['vit_base_patch16_224_moco']
    if pretrained:
        load_pretrained(
            model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3), filter_fn=_conv_filter, moco=True)
    return model


def vit_mae_patch16_224(pretrained=False, w_rel=False, **kwargs):
    if w_rel:
        model = VisionTransformerWRel(patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
                                      mae=True, **kwargs)
    else:
        model = VisionTransformer(patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
                                  norm_layer=partial(nn.LayerNorm, eps=1e-6), global_pooling=True, **kwargs)
    model.default_cfg = default_config['vit_mae_patch16_224']
    if pretrained:
        load_pretrained(
            model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3), filter_fn=_conv_filter, mae=True)
    return model


def vit_large_patch16_224(pretrained=False, w_rel=False, **kwargs):
    if w_rel:
        model = VisionTransformerWRel(patch_size=16, embed_dim=1024, depth=24, num_heads=16, mlp_ratio=4, qkv_bias=True,
                                      **kwargs)
    else:
        model = VisionTransformer(patch_size=16, embed_dim=1024, depth=24, num_heads=16, mlp_ratio=4, qkv_bias=True,
                                  norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    model.default_cfg = default_config['vit_large_patch16_224']
    if pretrained:
        load_pretrained(model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3))
    return model


## **Input arguments - input_arguments.py & Basic plotting utilities**

In [ ]:
import argparse


def get_arg_parser():
    parser = argparse.ArgumentParser(description='Training multi-class classifier')
    parser.add_argument('--arc', type=str, default='vgg', metavar='N',
                        help='Model architecture')
    parser.add_argument('--train_dataset', type=str, default='imagenet', metavar='N',
                        help='Testing Dataset')
    parser.add_argument('--method', type=str,
                        default='ours_c',
                        choices=['rollout', 'lrp', 'partial_lrp', 'transformer_attribution', 'attn_last_layer',
                                 'attn_gradcam', 'generic_attribution', 'ours', 'ours_c'],
                        help='')
    parser.add_argument('--thr', type=float, default=0.,
                        help='threshold')
    parser.add_argument('--start_layer', type=int, default=4,
                        help='start_layer')
    parser.add_argument('--K', type=int, default=1,
                        help='new - top K results')
    parser.add_argument('--save-img', action='store_true',
                        default=True,
                        help='')
    parser.add_argument('--no-ia', action='store_true',
                        default=False,
                        help='')
    parser.add_argument('--no-fx', action='store_true',
                        default=False,
                        help='')
    parser.add_argument('--no-fgx', action='store_true',
                        default=False,
                        help='')
    parser.add_argument('--no-m', action='store_true',
                        default=False,
                        help='')
    parser.add_argument('--no-reg', action='store_true',
                        default=False,
                        help='')
    parser.add_argument('--is-ablation', type=bool,
                        default=False,
                        help='')
    parser.add_argument('--len-lim', type=int,
                        default=1,
                        help='')
    parser.add_argument('--imagenet-seg-path', type=str, required=True)

    return parser


In [ ]:
def _ntuple(n):
    def parse(x):
        if isinstance(x, collections.abc.Iterable):
            return x
        return tuple(repeat(x, n))
    return parse


to_1tuple = _ntuple(1)
to_2tuple = _ntuple(2)
to_3tuple = _ntuple(3)
to_4tuple = _ntuple(4)
to_ntuple = _ntuple


def image_vizformat(img):
    inr = transforms.Normalize(mean=[-0.5/.5, -0.5/.5, -0.5/.5], std=[1/0.5, 1/0.5, 1/0.5])
    img = inr(img[0])
    img = torch.permute(img, (1, 2, 0))
    return img.detach().cpu().numpy()

def plot(image, isImage=True):
    if isImage:
        image = image_vizformat(image)
    plt.imshow(image)
    plt.axis("off")
    plt.show()
    
def plot_img_mask(image, mask):
    fig, axs = plt.subplots(nrows=1, ncols=2)
    axs[0].imshow(image_vizformat(image))
    axs[0].axis("off")
    
    axs[1].imshow(mask, cmap="gray")
    axs[1].axis("off")
    
    plt.show()
    
def plotside(i1, i2, cm1="autumn_r", cm2="autumn_r", title="Title", save_path=None):
    fig, axs = plt.subplots(nrows=1, ncols=2)
    plt.title(title)
    axs[0].imshow(i1, cmap=cm1)
    axs[1].imshow(i2, cmap=cm2)
    axs[0].axis("off")
    axs[1].axis("off")
    
    if save_path is None:
        plt.show()
    else:
        plt.savefig(save_path)

# <font color="red"> **Evaluation Metrics**</font>

In [ ]:
def calculate_iou(pred_mask, true_mask):
    # Ensure masks are binary (0 or 1)
    pred_mask = np.where(pred_mask > 0.5, 1, 0)
    true_mask = np.where(true_mask > 0.5, 1, 0)
    
    # Calculate Intersection and Union
    intersection = np.logical_and(pred_mask, true_mask).sum()
    union = np.logical_or(pred_mask, true_mask).sum()
    
    # Calculate IoU
    iou = intersection / union if union != 0 else 0
    
    return round(iou, 2)

# <font color="#DC143C"> **Building the Vision Transformer model**

## **Initializing the weight**

In [ ]:
def _no_grad_trunc_normal_(tensor, mean, std, a, b):
    # Cut & paste from PyTorch official master until it's in a few official releases - RW
    # Method based on https://people.sc.fsu.edu/~jburkardt/presentations/truncated_normal.pdf
    def norm_cdf(x):
        # Computes standard normal cumulative distribution function
        return (1. + math.erf(x / math.sqrt(2.))) / 2.

    if (mean < a - 2 * std) or (mean > b + 2 * std):
        warnings.warn("mean is more than 2 std from [a, b] in nn.init.trunc_normal_. "
                      "The distribution of values may be incorrect.",
                      stacklevel=2)

    with torch.no_grad():
        # Values are generated by using a truncated uniform distribution and
        # then using the inverse CDF for the normal distribution.
        # Get upper and lower cdf values
        l = norm_cdf((a - mean) / std)
        u = norm_cdf((b - mean) / std)

        # Uniformly fill tensor with values from [l, u], then translate to
        # [2l-1, 2u-1].
        tensor.uniform_(2 * l - 1, 2 * u - 1)

        # Use inverse cdf transform for normal distribution to get truncated
        # standard normal
        tensor.erfinv_()

        # Transform to proper mean, std
        tensor.mul_(std * math.sqrt(2.))
        tensor.add_(mean)

        # Clamp to ensure it's in the proper range
        tensor.clamp_(min=a, max=b)
        return tensor


def trunc_normal_(tensor, mean=0., std=1., a=-2., b=2.):
    # type: (Tensor, float, float, float, float) -> Tensor
    r"""Fills the input Tensor with values drawn from a truncated
    normal distribution. The values are effectively drawn from the
    normal distribution :math:`\mathcal{N}(\text{mean}, \text{std}^2)`
    with values outside :math:`[a, b]` redrawn until they are within
    the bounds. The method used for generating the random values works
    best when :math:`a \leq \text{mean} \leq b`.
    Args:
        tensor: an n-dimensional `torch.Tensor`
        mean: the mean of the normal distribution
        std: the standard deviation of the normal distribution
        a: the minimum cutoff value
        b: the maximum cutoff value
    Examples:
        >>> w = torch.empty(3, 5)
        >>> nn.init.trunc_normal_(w)
    """
    return _no_grad_trunc_normal_(tensor, mean, std, a, b)

## **Patch Embeddings**

In [ ]:
class PatchEmbed(nn.Module):
    """ Image to Patch Embedding
    """

    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, C, H, W = x.shape
        # FIXME look at relaxing size constraints
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x


## **MLP**

In [ ]:
class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

## **Attention**

In [ ]:
class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5  # This is root_over(dh) with which we divided Q.KT

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)  # Attention's multiplication. That is, we have to pass the attention output through a Linear layer. That is the linear layer.
        self.proj_drop = nn.Dropout(proj_drop)

        self.attn_gradients = None
        self.attention_map = None

        self.vproj = None
        self.input = None

    def save_input(self, z):
        self.input = z

    def get_input(self):
        return self.input

    # Backward hook. Therefore, param is automatically gradients for that layer.
    def save_attn_gradients(self, attn_gradients):
        self.attn_gradients = attn_gradients

    def get_attn_gradients(self):
        return self.attn_gradients

    def save_attention_map(self, attention_map):
        self.attention_map = attention_map

    def get_attention_map(self):
        return self.attention_map

    def forward(self, x, register_hook=False):
        b, n, _, h = *x.shape, self.num_heads
        self.save_input(x)
        
        print("\n\n\n ================= Inside attention block ================ \n\n")
        
        qkv = self.qkv(x)
        q, k, v = rearrange(qkv, 'b n (qkv h d) -> qkv b h n d', qkv=3, h=h)
        
        print(f"qkv shape: {qkv.shape}\n\t q: {q.shape}\n\t k: {k.shape}\n\t v:{v.shape}\n\tweight of qkv: {self.qkv.weight.shape}")
        
        dots = torch.einsum('bhid,bhjd->bhij', q, k) * self.scale
        
        attn = dots.softmax(dim=-1)
        attn = self.attn_drop(attn)
    
        out = torch.einsum('bhij,bhjd->bhid', attn, v)  # Simple torch.matmul actually.
        
        print(f"dots : {dots.shape}")
        print(f"attn i.e. softmax of dots: {attn.shape}")
        print(f"out shape: {out.shape}  | attn * v -> bhij,bhjd->bhid")

        self.save_attention_map(attn)
        if register_hook:
            attn.register_hook(self.save_attn_gradients)

        out = rearrange(out, 'b h n d -> b n (h d)')
        
        print(f"out after rearrange: {out.shape} | b h n d -> b n (h d)")

        out = self.proj(out)
        out = self.proj_drop(out)
        

        self.vproj = torch.matmul(rearrange(v, 'b h n d -> b n (h d)'), self.proj.weight.t())  # v is multipled with proj weight even for the attention output. We can see in out=self.proj(out) line.
        # The thing is that, v was not previously multiplied with proj. instead we multipled the whole result. The paper says v is multipled with proj. Therefore, it is done here.

        print(f"out after proj: {out.shape} | W_proj shape: {self.proj.weight.shape}")
        print(f"vproj after proj: {self.vproj.shape}\n")
        print("================ End of attn block===============\n")
        
        
        return out

## **Encoder block**

In [ ]:
class Block(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, drop=0., attn_drop=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, block_index=None):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(dim, num_heads=num_heads, qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=drop)
        self.norm2 = norm_layer(dim)

        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)  # hidden_features is the output layer's neuron count.

        self.input = None
        self.output = None
        self.tilde = None
        
        self.block_index = block_index

    def save_input(self, z):
        self.input = z

    def get_input(self):
        return self.input

    def save_tilde(self, z):
        self.tilde = z

    def get_tilde(self):
        return self.tilde

    def save_output(self, z):
        self.output = z

    def get_output(self):
        return self.output

    def forward(self, x, register_hook=False):
        print("\n\n\n___________________________________________________________________\n")
        print(f"Log for block: {self.block_index}\n\n")
        
        self.save_input(x)
        print(f"shape of x: {x.shape}")
        
        attn_out = self.attn(self.norm1(x), register_hook=register_hook)
        out1 = x + attn_out
        self.save_tilde(out1)
        
        
        mlp_out = self.mlp(self.norm2(out1))
        out = out1 + mlp_out
        self.save_output(out)
        
        print(f"attn_out: {attn_out.shape} | out1 = x + attn_out: {out1.shape}")
        print(f"mlp_out: {mlp_out.shape} | out = out1 + mlp_out: {out.shape}")
        

        return out

## <font color="#A02334"> **Vision Transformer Model** </font>

In [ ]:

class VisionTransformer(nn.Module):
    """ Vision Transformer
    """

    def __init__(self, img_size=224, patch_size=16, in_chans=3, num_classes=1000, embed_dim=768, depth=12,
                 num_heads=12, mlp_ratio=4., qkv_bias=False, drop_rate=0., attn_drop_rate=0., norm_layer=nn.LayerNorm,
                 global_pooling=False, dino=False):
        super().__init__()
        self.global_pooling = global_pooling
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models
        self.patch_embed = PatchEmbed(
            img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim)
        num_patches = self.patch_embed.num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)

        self.blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias,
                drop=drop_rate, attn_drop=attn_drop_rate, norm_layer=norm_layer, block_index=i)
            for i in range(depth)])
        if global_pooling:
            self.fc_norm = norm_layer(embed_dim)
        else:
            self.norm = norm_layer(embed_dim)

        # Classifier head
        if dino:
            self.head = nn.Linear(embed_dim * 2, num_classes) if num_classes > 0 else nn.Identity()
        else:
            self.head = nn.Linear(embed_dim, num_classes) if num_classes > 0 else nn.Identity()

        self.cls_gradients = None
        self.input_grad = None
        self.dino = dino

        trunc_normal_(self.pos_embed, std=.02)
        trunc_normal_(self.cls_token, std=.02)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'pos_embed', 'cls_token'}

    def save_cls_gradients(self, cls_gradients):
        self.cls_gradients = cls_gradients

    def get_cls_gradients(self):
        return self.cls_gradients

    def save_input_gradients(self, input_grad):
        self.input_grad = input_grad

    def get_input_gradients(self):
        return self.input_grad

    def forward(self, x, register_hook=False):
        B = x.shape[0]
        #         if register_hook:
        #             x.register_hook(self.save_input_gradients)

        x = self.patch_embed(x)

        cls_tokens = self.cls_token.expand(B, -1, -1)  # stole cls_tokens impl from Phil Wang, thanks
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)

        for blk in self.blocks[:-1]:
            x = blk(x, register_hook=register_hook)

        x.register_hook(self.save_cls_gradients)

        x = self.blocks[-1](x, register_hook=register_hook)

        if self.global_pooling:
            x = x[:, 1:, :].mean(axis=1)
            x = self.fc_norm(x)
        elif self.dino:
            x = self.norm(x)
            x = torch.cat((x[:, 0].unsqueeze(-1), x[:, 1:, :].mean(axis=1).unsqueeze(-1)), dim=-1)
            x = x.reshape(x.shape[0], -1)
        else:
            x = self.norm(x)
            x = x[:, 0]

        x = self.head(x)
        return x

# <br><br><center><font size="+6"> **Run and Test** </font><br><br>

In [ ]:
data_path = "/kaggle/input/imagenet-seg-5000/gtsegs_ijcv.mat"
data_length = 2
batch_size = 1
num_workers = 7

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])
label_transform = transforms.Compose([transforms.Resize((224, 224), Image.NEAREST), ])

In [ ]:
dataset = ImagenetSegLoader(data_path, data_length, transform=image_transform, target_transform=label_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=False)
dataloader = tqdm(dataloader)  # Would help tracking loop iteration along with setting some verbose text.

In [ ]:
images, masks = [], []

for img, mask in dataloader:
    images.append(img.to(device))
    masks.append(mask.to(device))

In [ ]:
model = vit_base_patch16_224_dino(pretrained=True).to(device)

The model have 12 blocks and 12 heads. As 12 blocks i.e. 12 layers, therefore, attention is called 12 times and hence output printed for 12 times.

In [ ]:
idx = 0
x = images[idx]
output = model(x, register_hook=True)

# clear_output()

In [ ]:
#@title - Gradient Visualization

# index = np.argmax(output.cpu().data.numpy(), axis=-1)
# out_sum = torch.sum(output)

# one_hot = np.zeros((batch_size, output.size()[-1]), dtype=np.float32)
# one_hot[np.arange(batch_size), index] = 1
# one_hot = torch.from_numpy(one_hot).requires_grad_(True).to(device)
# one_hot = torch.sum(one_hot)

# def grad_accumulator():
#     gradients = torch.eye(197, 197)
#     for blk in model.blocks:
#         grd = blk.attn.get_attn_gradients()
#         grd = grd.reshape(-1, grd.shape[-2], grd.shape[-1]).mean(0)
#         grd = torch.maximum(torch.zeros_like(grd), grd)  # Only positive values
#         gradients = torch.mm(gradients, grd) + gradients
#     return gradients, grd # Last layer's gradient is in grd


# def grad_hmp(grd):
#     grad_map = grd[1:, 1:].clamp(min=0).mean(0).reshape(14, 14).detach().cpu().numpy()
#     grad_map = cv2.resize(grad_map, mask.shape, interpolation = cv2.INTER_NEAREST)
#     thres_val = grad_map.mean()
#     _, grad_map = cv2.threshold(grad_map, thres_val, 255, cv2.THRESH_BINARY)
#     plt.imshow(grad_map, cmap="autumn_r")
#     plt.title("Accumulated attention gradients")
#     plt.colorbar()
#     plt.axis("off")

"""
    Calculating Positive Gradients
"""

# model.zero_grad()
# one_hot.backward(retain_graph=True)
# pos_grads = grad_accumulator() 
# grad_hmp(pos_grads)

"""
    
    Calculating negative gradients

"""

# model.zero_grad()

# # Sum of non-class probs = 1 - current_onehot as curr_onehot = softmax(output[max_idx])
# one_hot = out_sum - one_hot
# one_hot.backward(retain_graph=True)

# neg_gradients = grad_accumulator()
# grad_hmp(neg_gradients)


"""
    Difference between positive and negative hmp
"""

# diff_grad_map = grad_map - ngrad_map
# plt.imshow(diff_grad_map, cmap="autumn_r")
# plt.colorbar()

# **Beyond Intuition tokenwise method**

In [ ]:
import torch
import numpy as np


def beyond_intuition_tokenwise(model, x, device, onehot=None, index=None, steps=20, start_layer=1, samples=20, noise=0.1, mae=False, ssl=False, dino=False, taken_head_idx = None):

    # A dictionary to track down all the output at every step, so we can visualize it.
    stat_dict = {
        "input": x,
        "steps": steps,
        "start_layer": start_layer,
    }

    b = x.shape[0]  # Batch size
    x = x.to(device)

    output = model(x)
    
    index = np.argmax(output.cpu().data.numpy(), axis=-1)
    out_sum = torch.sum(output)

    one_hot = np.zeros((batch_size, output.size()[-1]), dtype=np.float32)
    one_hot[np.arange(batch_size), index] = 1
    one_hot = torch.from_numpy(one_hot).requires_grad_(True).to(device)
    one_hot = torch.sum(one_hot)

    model.zero_grad()
    one_hot.backward(retain_graph=True)

    _, num_head, num_tokens, _ = model.blocks[-1].attn.get_attention_map().shape
    R = torch.eye(num_tokens, num_tokens).expand(b, num_tokens, num_tokens).to(device)
    
    stat_dict['final_blk_attn_shape'] = model.blocks[-1].attn.get_attention_map().shape
    stat_dict['pre_attn_perception_R'] = R
    stat_dict['attention_perception'] = dict()


    for blk_idx, blk in enumerate(model.blocks):
        if blk_idx < start_layer - 1:
            continue

        # Calculate alpha in the paper.
        z = blk.get_input()  # We can even call it z.
        vproj = blk.attn.vproj  # vproj = Z * Wv * W(l).
        order = torch.linalg.norm(vproj, dim=-1).squeeze() / torch.linalg.norm(z, dim=-1).squeeze()  # Z * Wv * W(l) / Z    --->  V / Z.  # Shape --> (197,)
        m = torch.diag_embed(order)  # Converted the order into a diagonal matrix. # Shape (197, 197)

        # Get attention map A_i which will be multiplied by alpha_i.
        cam = blk.attn.get_attention_map()  # Shape --> (batch_size, num_heads, num_tokens, num_tokens)   (12, 197, 197)
        cam = cam.reshape(-1, cam.shape[-2], cam.shape[-1])  # Make the shape (num_heads, num_tokens, num_tokens).  

        
        if taken_head_idx is None:
            cam = cam.mean(0)  # Then take headwise mean and make it of shape (num_tokens, num_tokens).
            # That is, take average of all head's attention for a token.
        else:
            cam = cam[taken_head_idx]  # Select specific head only.

        # Also, try taking variance of gradient of attention and multiply then with that.
        O_t = torch.matmul(cam.to(device), m.to(device))  # O = AZW   >>>  cam shape (197, 197) |  m shape (197, 197)
        R = R + torch.matmul(O_t, R.to(device))

        
        stat_dict['attention_perception'][f"rOps_{blk_idx}"] = {
            "z": z,
            "m": m,
            "order": order,
            "vproj": vproj,
            "cam_noreshape": cam,
            "mean_cam": cam.mean(0),  # Headwise mean
            "O": O_t,
            "R": R,
        }


    if ssl:
        if mae:
            return R[:, 1:, 1:].abs().mean(axis=1)
        elif dino:
            return R[:, 1:, 1:].abs().mean(axis=1) + R[:, 0, 1:].abs()
        else:
            return R[:, 0, 1:].abs()

    stat_dict["R_attn_perception"] = R
    stat_dict['reasoning_feedback'] = dict()

    total_gradients = torch.zeros(b, num_head, num_tokens, num_tokens, device=device)
    for alpha in np.linspace(0, 1, steps):
        data_scaled = x * alpha
        output = model(data_scaled, register_hook=True)
        one_hot = np.zeros((b, output.size()[-1]), dtype=np.float32)
        one_hot[np.arange(b), index] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True).to(device)
        one_hot = torch.sum(one_hot * output)

        model.zero_grad()
        one_hot.backward(retain_graph=True)

        gradients = model.blocks[-1].attn.get_attn_gradients()
        total_gradients += gradients

        stat_dict['reasoning_feedback'][f"alpha_{alpha}"] = {
            "data_scaled": data_scaled,
            "output": output,
            "one_hot": one_hot,
            "gradients": gradients,
            "gradient_shape":  model.blocks[-1].attn.get_attn_gradients().shape,
            "total_gradients": total_gradients,
        }

    W_state = (total_gradients / steps).clamp(min=0).mean(1).reshape(b, num_tokens, num_tokens)
    R = W_state * R.abs()

    stat_dict["W_state"] = W_state
    stat_dict["R_resoaning_feedback"] = R


    if mae:
        R_MAE = R[:, 1:, 1:].mean(axis=1)
        stat_dict["R"] = (R_MAE, "MAE")
        return R_MAE, stat_dict

    elif dino:
        R_DINO = (R[:, 1:, 1:].mean(axis=1) + R[:, 0, 1:])
        stat_dict["R"] = (R_DINO, "DINO")
        return R_DINO, stat_dict

    else:
        R_general = R[:, 0, 1:]
        stat_dict["R"] = (R_general, "General Case")
        return R_general, stat_dict

# **Attribution result vs Mask - Test for Tokenwise method's weakness**

In [ ]:
def attr_vs_mask(range_lim=None):
    os.makedirs("/kaggle/working/attn_vs_mask", exist_ok=True)
    for idx in tqdm(range(range_lim)):
        x = images[idx]
        heatmap, _ = beyond_intuition_tokenwise(model, x, device, dino=True, steps=30)
        
        # Heatmap processing
        hmp = heatmap.reshape(14, 14).detach().cpu().numpy()
        hmp = cv2.resize(hmp, (224, 224), interpolation = cv2.INTER_NEAREST)  # reshape to 224, 224

        # Mask
        mask = masks[idx]
        mask = mask[0].detach().cpu()
        
        iou_score = calculate_iou(hmp, mask.detach().cpu())  # Calculate iou score.
        plotside(hmp, mask, title=f"IOU score: {iou_score}", save_path=f"attn_vs_mask/{idx}.png")